In [2]:
import pandas as pd
import numpy as np
from pprint import pprint as pp
p = print

BASE_PATH = '../Raw data/'

# Users

In [5]:
users = pd.read_csv(f'{BASE_PATH}/users.csv')
users.set_index('id', inplace=True)
#users.rename(columns={'accessibility_needs': 'special_needs'}, inplace=True)
#users.drop(columns=['address'], inplace = True)
users

,name,age,domicile_location,residence_location,work_location,edu_type,occupation,special_needs
id,,,,,,,,
1,Mario,13,Via della Lovara 9 38012 Mollaro TN,Via della Lovara 9 38012 Mollaro TN,Via Alpini 17 38017 Mezzolombardo TN,MS,Stud.,NaN
2,Jessica,16,Via alla Cascata 2 38123 Povo TN,Via alla Cascata 2 38123 Povo TN,Via Zambra 3 38121 Trento TN,HS,Stud.,NaN
3,Luca,18,Via Luigi Einaudi 60 38122 Trento TN,Via Luigi Einaudi 60 38122 Trento TN,Via Cristoforo Madruzzo 24 38122 Trento TN,HS,Stud.,Access.
4,Gian,20,Via Giuseppe Mazzini 35 38122 Trento TN,Via Giuseppe Mazzini 35 38122 Trento TN,Corso Bettini 84 Rovereto 38068 TN,U,Stud.,NaN
5,Gaia,22,Via della Malpensada 140 38123 Trento TN,Via Giovanni Amendola 15 Bologna 40121 BO,Via Giuseppe Verdi 26 38122 Trento,U,Stud.,NaN
6,Carla,22,Via della Malpensada 140 38123 Trento TN,Corso Inghilterra 27 Torino 10138 TO,Via Sommarive 9 Povo 38123 TN,U,Stud.,Space
7,Fausto,32,Calle dei Bergamaschi 568 30135 Venezia VE,Calle dei Bergamaschi 568 30135 Venezia VE,Via Sommarive 14 38123 Povo TN,U,Prof.,NaN
8,Lucia,43,Case Ferrovieri Porta Nuova 37138 Verona VR,Case Ferrovieri Porta Nuova 37138 Verona VR,Via Vigilio Inama 5 Trento 38122 TN,U,Prof.,TimeOpt.
9,Anna,52,Via Gustavo Modena 32 38065 Mori TN,Via Gustavo Modena 32 38065 Mori TN,Via Santissima Trinita' 38 38122 Trento TN,MS,Janitor,NaN


## Users schedule (calendar + exceptions)

In [58]:
weekday_to_index = {
  'Monday': 0,
  'Tuesday': 1,
  'Wednesday': 2,
  'Thursday': 3,
  'Friday': 4,
  'Saturday': 5,
  'Sunday': 6
}

def weekdays_to_binary(weekdays):
  s = '0' * 7
  weekdays = weekdays.split(' to ')
  if len(weekdays) == 1:
    i = weekday_to_index.get(weekdays[0])
    s = s[:i] + '1' + s[i + 1:]
  else:
    for i in range(weekday_to_index.get(weekdays[0]), weekday_to_index.get(weekdays[1]) + 1):
      s = s[:i] + '1' + s[i + 1:]
  return s

users_schedule = pd.read_csv(f'{BASE_PATH}/users_schedule.csv')
users_schedule['weekdays'] = users_schedule['weekdays'].apply(weekdays_to_binary)

distinct_weekdays = users_schedule['weekdays'].unique()
user_calendars = pd.DataFrame({
  'weekdays': distinct_weekdays,
  'start_date': ['20230913'] * len(distinct_weekdays), # 13 settembre 2023 found from google
  'end_date': ['20240610'] * len(distinct_weekdays) # 10 giugno 2024 found from google
})
user_calendars.index = user_calendars.index.to_series().apply(lambda x: 'c_' + str(x))
user_calendars.index.set_names('id', inplace=True)

users_schedule

,user_name,from,to,weekdays,arrive_before,leave_after,info
0,Mario,Mollaro,Mezzolombardo,1111100,08:00:00 AM,01:00:00 PM,Train
1,Jessica,Sanba,Trento Center,1111110,08:00:00 AM,01:00:00 PM,Bus
2,Luca,South of Trento,Trento Center,1111110,08:00:00 AM,01:00:00 PM,Wheelchair
3,Gian,Center of Trento,Rovereto,1110000,10:30:00 AM,02:30:00 PM,University student
4,Gian,Center of Trento,Rovereto,0001100,10:30:00 AM,04:30:00 PM,University student
5,Gaia,Trento University,BUC,0000100,08:30:00 PM,NaN,Varied schedule
6,Lucia,Verona,Povo,1111111,08:30:00 AM,01:30:00 AM,Commuter
7,Anna,Mori,Povo Middle-School,1111100,01:30:00 PM,07:00:00 PM,Janitor
8,Gaia,Bologna Centrale,Trento,0000100,NaN,05:00:00 PM,Return to Trento by Sunday night before 9PM


In [59]:
user_calendars

,weekdays,start_date,end_date
id,,,
c_0,1111100,20230913,20240610
c_1,1111110,20230913,20240610
c_2,1110000,20230913,20240610
c_3,0001100,20230913,20240610
c_4,0000100,20230913,20240610
c_5,1111111,20230913,20240610


In [60]:
users_schedule_exceptions = pd.read_csv(f'{BASE_PATH}/users_schedule_exceptions.csv')

dates = pd.to_datetime(users_schedule_exceptions['date']).dt.strftime('%Y%m%d').astype(str)

users_calendar_exceptions = pd.DataFrame({
  'date': dates,
  'exception_type': [1] * len(dates) # 1 = commitment has been added (2 for removed)
})
users_calendar_exceptions.index = users_calendar_exceptions.index.to_series().apply(lambda x: 'ce_' + str(x))
users_calendar_exceptions.index.set_names('id', inplace=True)

users_schedule_exceptions

,user_name,from,to,date,arrive_before,leave_after,info
0,Carla,Trento,Turin,2023-10-31,01:00:00 PM,NaN,Luggage
1,Fausto,Venice S. Lucia,Povo,2023-12-21,09:00:00 AM,NaN,Professor


In [61]:
users_calendar_exceptions

,date,exception_type
id,,
ce_0,20231031,1
ce_1,20231221,1


## Users commitments (same as GTFS Trip)

In [62]:
c_users = users.merge(users_schedule, left_on='name', right_on='user_name', how='inner').index.to_list()
c_calendar = list(map(lambda x: 'c_' + str(x), users_schedule.index.to_list()))

ce_users = users.merge(users_schedule_exceptions, left_on='name', right_on='user_name', how='inner').index.to_list()
ce_calendar = list(map(lambda x: 'ce_' + str(x), users_schedule_exceptions.index.to_list()))

users_commitments = pd.DataFrame({
  'user': ce_users + c_users,
  'calendar': ce_calendar + c_calendar,
  'from': users_schedule_exceptions['from'].tolist() + users_schedule['from'].tolist(),
  'to': users_schedule_exceptions['to'].tolist() + users_schedule['to'].tolist(),
  'arrive_before': users_schedule_exceptions['arrive_before'].tolist() + users_schedule['arrive_before'].tolist(),
  'leave_after': users_schedule_exceptions['leave_after'].tolist() + users_schedule['leave_after'].tolist()
})
users_commitments['arrive_before'] = pd.to_datetime(users_commitments['arrive_before'], format='%I:%M:%S %p').dt.strftime('%H:%M:%S')
users_commitments['leave_after'] = pd.to_datetime(users_commitments['leave_after'], format='%I:%M:%S %p').dt.strftime('%H:%M:%S')
users_commitments

,user,calendar,from,to,arrive_before,leave_after
0,0,ce_0,Trento,Turin,13:00:00,NaN
1,1,ce_1,Venice S. Lucia,Povo,09:00:00,NaN
2,0,c_0,Mollaro,Mezzolombardo,08:00:00,13:00:00
3,1,c_1,Sanba,Trento Center,08:00:00,13:00:00
4,2,c_2,South of Trento,Trento Center,08:00:00,13:00:00
5,3,c_3,Center of Trento,Rovereto,10:30:00,14:30:00
6,4,c_4,Center of Trento,Rovereto,10:30:00,16:30:00
7,5,c_5,Trento University,BUC,20:30:00,NaN
8,6,c_6,Verona,Povo,08:30:00,01:30:00
9,7,c_7,Mori,Povo Middle-School,13:30:00,19:00:00


# Transportation

## Convert scrapy results to GTMS

In [138]:
SCRAPY_OUT_BASE_PATH = '../Raw data/'
SCRAPY_BASEPATH = './E656_scraper/'
SCRAPY_DFS = {}
SCRAPY_FILENAMES = {
  'trento_trains': 'trento_trains',
  'ftm_trains': 'ftm_routes_trips',
  'ftm_stops': 'ftm_stops',
  # 'stops': 'stops',
}
for name, path in SCRAPY_FILENAMES.items():
  SCRAPY_DFS[name] = pd.read_json(SCRAPY_BASEPATH + path + '.json')

### FTM

In [130]:
ftm_trips = SCRAPY_DFS['ftm_trains'].copy()
ftm_trips = ftm_trips[ftm_trips['item_type'] == 'Train']
ftm_trips.drop(columns=['item_type', 'url', 'train_name', 'stop', 'arrival_time', 'departure_time'], inplace=True)
ftm_trips.dropna(inplace=True)

ftm_stop_times = SCRAPY_DFS['ftm_trains'].copy()
ftm_stop_times = ftm_stop_times[ftm_stop_times['item_type'] == 'TripStop']
ftm_stop_times.drop(columns=['item_type', 'url', 'name', 'calendar'], inplace=True)
# ftm_trips.dropna(inplace=True)
# ftm_trips['name'] = ftm_trains['name'].astype(int)
# ftm_trips['calendar'] = ftm_trains['calendar'].astype(str).apply(lambda x: 'ftm_calendar_' + x[-3])
ftm_stop_times = pd.merge(ftm_trips, ftm_stop_times, left_on='name', right_on='train_name', how='inner')

def find_train_direction (name):
  tmp = ftm_stop_times.loc[ftm_stop_times.isnull().any(axis=1)]
  tmp = tmp.loc[ftm_stop_times['name'] == name]
  stop = tmp.loc[tmp['arrival_time'].isnull()].drop_duplicates()['stop'].values[0]
  if stop == 'Trento Ftm':
    return 0
  else:
    return 1
    
  
ftm_stop_times['name'] = ftm_stop_times['name'].astype(int)
ftm_stop_times['train_name'] = ftm_stop_times['train_name'].astype(int)

ftm_trips['name'] = ftm_trips['name'].astype(int)
ftm_trips['calendar'] = ftm_trips['calendar'].astype(str).apply(lambda x: 'ftm_calendar_' + x[-3])
ftm_trips['direction'] = ftm_trips['name'].apply(find_train_direction)

ftm_trips.drop_duplicates(inplace=True)

### Trento

In [143]:
trips = SCRAPY_DFS['trento_trains'].copy()
trips = trips[trips['item_type'] == 'Train']
trips.drop(columns=['item_type', 'url', 'train_name', 'stop', 'arrival_time', 'departure_time'], inplace=True)
# trips.dropna(inplace=True)

stop_times = SCRAPY_DFS['trento_trains'].copy()
stop_times = stop_times[stop_times['item_type'] == 'TripStop']
stop_times.drop(columns=['item_type', 'url', 'name'], inplace=True)
# ftm_trips.dropna(inplace=True)
# ftm_trips['name'] = ftm_trains['name'].astype(int)
# ftm_trips['calendar'] = ftm_trains['calendar'].astype(str).apply(lambda x: 'ftm_calendar_' + x[-3])
# ftm_stop_times = pd.merge(ftm_trips, ftm_stop_times, left_on='name', right_on='train_name', how='inner')

# def find_train_direction (name):
#   tmp = ftm_stop_times.loc[ftm_stop_times.isnull().any(axis=1)]
#   tmp = tmp.loc[ftm_stop_times['name'] == name]
#   stop = tmp.loc[tmp['arrival_time'].isnull()].drop_duplicates()['stop'].values[0]
#   if stop == 'Trento Ftm':
#     return 0
#   else:
#     return 1
    
  
# ftm_stop_times['name'] = ftm_stop_times['name'].astype(int)
# ftm_stop_times['train_name'] = ftm_stop_times['train_name'].astype(int)

# ftm_trips['name'] = ftm_trips['name'].astype(int)
# ftm_trips['calendar'] = ftm_trips['calendar'].astype(str).apply(lambda x: 'ftm_calendar_' + x[-3])
# ftm_trips['direction'] = ftm_trips['name'].apply(find_train_direction)

# ftm_trips.drop_duplicates(inplace=True)

trips

,name
0,16105
1,16164
2,16107
3,3461
4,8505
...,...
78,TN102
79,8954
80,23021
81,16932


## Write stops

## Write trips

In [111]:
ftm_trips_ = pd.DataFrame({
  'route_id': ['ftm_route_1'] * len(ftm_trips),
  'service_id': ftm_trips['calendar'].to_list(),
  'trip_id': map(lambda x: f'ftm_train_{x}', ftm_trips['name'].to_list()),
  'trip_headsign': ftm_trips['name'].to_list(),
  'trip_short_name': ftm_trips['name'].to_list(),
  'direction_id': ftm_trips['direction'].to_list()
})
ftm_trips_.to_csv(f'{SCRAPY_OUT_BASE_PATH}tt_ftm/trips.txt')
ftm_trips_

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id
0,ftm_route_1,ftm_calendar_1,ftm_train_302,302,302,0
1,ftm_route_1,ftm_calendar_1,ftm_train_301,301,301,1
2,ftm_route_1,ftm_calendar_1,ftm_train_4,4,4,0
3,ftm_route_1,ftm_calendar_1,ftm_train_1,1,1,1
4,ftm_route_1,ftm_calendar_1,ftm_train_6,6,6,0
5,ftm_route_1,ftm_calendar_1,ftm_train_303,303,303,1
6,ftm_route_1,ftm_calendar_2,ftm_train_51,51,51,1
7,ftm_route_1,ftm_calendar_1,ftm_train_3,3,3,1
8,ftm_route_1,ftm_calendar_1,ftm_train_305,305,305,1
9,ftm_route_1,ftm_calendar_1,ftm_train_8,8,8,0


## Write trip stops

In [136]:
stop_times = []
ftm_stop_times.drop_duplicates(inplace=True)
ftm_stop_times.sort_values(by=['name', 'arrival_time'], ascending=[True, True], na_position='first', inplace=True)

for group_name, group_data in ftm_stop_times.groupby('name'):
  stop_times = stop_times + list(range(1, len(group_data) + 1))
  
ftm_stop_times_ = pd.DataFrame({
  'trip_id': map(lambda x: f'ftm_train_{x}', ftm_stop_times['name'].to_list()),
  'arrival_time': map(lambda x: x.replace('.', ':') if x is not None else x, ftm_stop_times['arrival_time'].to_list()),
  'departure_time':map(lambda x: x.replace('.', ':') if x is not None else x, ftm_stop_times['departure_time'].to_list()),
  'stop_id': ftm_stop_times['stop'].to_list(), # TODO link with stop
  'stop_sequence': stop_times
})
ftm_stop_times_.to_csv(f'{SCRAPY_OUT_BASE_PATH}tt_ftm/stop_times.txt')
ftm_stop_times_


,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,ftm_train_1,None,05:28:00,Male',1
1,ftm_train_1,05:30:00,05:31:00,Terzolas,2
2,ftm_train_1,05:33:00,05:34:00,Caldes,3
3,ftm_train_1,05:35:00,05:35:00,Cavizzana,4
4,ftm_train_1,05:37:00,05:37:00,Cassana,5
...,...,...,...,...,...
1681,ftm_train_319,12:58:00,12:59:00,Zambana,6
1682,ftm_train_319,13:01:00,13:02:00,Lavis,7
1683,ftm_train_319,13:07:00,13:08:00,Gardolo,8
1684,ftm_train_319,13:10:00,13:10:00,Trento Nord-zona Commerciale,9


## Load all needed static files

In [63]:
FILENAMES = {
  'agencies': f'{BASE_PATH}agencies.csv'
}

# Base folder for GTFS datasets
FOLDERS = [
  'tt_extraurban',
  'tt_urban',
  'flixbus',
  'trenitalia/lombardia',
  'trenitalia/piemonte'
  # 'trenitalia/toscana'
]

GTFS_FILES = [
  'calendar',
  'calendar_dates',
  'routes',
  'stops',
  'stop_times',
  'trips'
] # , 'transfers'

# Trentino Trasporti
for folder in FOLDERS:
  for gtfs_file in GTFS_FILES:
    FILENAMES[f'{folder}_{gtfs_file}'] = f'{BASE_PATH}{folder}/{gtfs_file}.txt'

# Loading csv files
DFS = {}
for name, path in FILENAMES.items():
  DFS[name] = pd.read_csv(path)

FILENAMES

{'agencies': '../Raw data/agencies.csv',
 'tt_extraurban_calendar': '../Raw data/tt_extraurban/calendar.txt',
 'tt_extraurban_calendar_dates': '../Raw data/tt_extraurban/calendar_dates.txt',
 'tt_extraurban_routes': '../Raw data/tt_extraurban/routes.txt',
 'tt_extraurban_stops': '../Raw data/tt_extraurban/stops.txt',
 'tt_extraurban_stop_times': '../Raw data/tt_extraurban/stop_times.txt',
 'tt_extraurban_trips': '../Raw data/tt_extraurban/trips.txt',
 'tt_urban_calendar': '../Raw data/tt_urban/calendar.txt',
 'tt_urban_calendar_dates': '../Raw data/tt_urban/calendar_dates.txt',
 'tt_urban_routes': '../Raw data/tt_urban/routes.txt',
 'tt_urban_stops': '../Raw data/tt_urban/stops.txt',
 'tt_urban_stop_times': '../Raw data/tt_urban/stop_times.txt',
 'tt_urban_trips': '../Raw data/tt_urban/trips.txt',
 'flixbus_calendar': '../Raw data/flixbus/calendar.txt',
 'flixbus_calendar_dates': '../Raw data/flixbus/calendar_dates.txt',
 'flixbus_routes': '../Raw data/flixbus/routes.txt',
 'flixbus_st

## Generate Calendars

In [64]:
CALENDAR_COLS = ['weekdays', 'start_date', 'end_date']
calendars = pd.DataFrame(columns=CALENDAR_COLS)

for folder in FOLDERS:
  df = DFS[f'{folder}_calendar'].copy()
  weekdays = df.loc[:, 'monday':'sunday'].astype(str).agg(''.join, axis='columns')
  if len(weekdays) > 0:
    df['weekdays'] = weekdays
    df.rename(columns={'service_id': 'id'}, inplace=True)
    df.set_index('id', inplace=True)
    df.drop(columns=df.loc[:, 'monday':'sunday'].columns, inplace=True)
    # p(df)
    calendars = pd.concat([calendars, df])

if not calendars.index.is_unique:
  p('WARNING: duplicate id values!!')
  p(calendars.loc[calendars.index.duplicated()])

calendars.index.set_names('id', inplace=True)
calendars

,weekdays,start_date,end_date
id,,,
12023091120240621,1111111,20230911,20240621
12023091820240621,1111111,20230918,20240621
12023092820240621,1111111,20230928,20240621
12023100420240621,1111111,20231004,20240621
22023091120231015,1111110,20230911,20231015
...,...,...,...
249212715855674,1111111,20230728,20231231
249212915855675,1111111,20230728,20231231
249568015857568,0000011,20230728,20231231


### Calendar exceptions
The calendar_dates.txt table explicitly activates or disables service by date.

`exception_type` Enum	Required	Indicates whether service is available on the date specified in the date field. Valid options are:
- 1 - Service has been added for the specified date.
- 2 - Service has been removed for the specified date.

TODO: double check the meaning of service_id

In [65]:
CALENDAR_EXCEPTIONS_COLS = ['date', 'type']
calendar_exceptions = pd.DataFrame(columns=CALENDAR_EXCEPTIONS_COLS)

for folder in FOLDERS:
  df = DFS[f'{folder}_calendar_dates'].copy()
  # p(df)
  df.rename(columns={'service_id': 'id', 'exception_type': 'type'}, inplace=True)
  df.set_index('id', inplace=True)
  calendar_exceptions = pd.concat([calendar_exceptions, df])

# if not calendar_exceptions.index.is_unique:
#   p('WARNING: duplicate id values!!')
#   p(calendar_exceptions.loc[calendar_exceptions.index.duplicated()])

calendar_exceptions.index.set_names('id', inplace=True)
calendar_exceptions

,date,type
id,,
22023091120240621,20231101,2
22023091120240621,20231208,2
22023091120240621,20231225,2
22023091120240621,20231226,2
22023091120240621,20240101,2
...,...,...
ITRP-TS200_6501_63235_20221211_20231209,20231111,1
ITRP-TS200_6501_63235_20221211_20231209,20231118,1
ITRP-TS200_6501_63235_20221211_20231209,20231125,1


## Generate Bus Stops + Places

Notes:
- Extraurban has "wheelchair_boarding"

In [66]:
PLACES_COLS = ['address', 'latitude', 'longitude']
places = pd.DataFrame(columns=PLACES_COLS)

STOPS_COLS = ['name', 'type', 'place']
stops = pd.DataFrame(columns=STOPS_COLS)

for folder in FOLDERS:
  df = DFS[f'{folder}_stops'].copy()
  df['stop_id'] = df['stop_id'].astype(str)
  df.drop(columns=['stop_code', 'stop_desc', 'zone_id'], inplace=True, errors='ignore')
  df.rename(columns={'stop_id': 'id'}, inplace=True)
  df['place'] = df['id']
  df.set_index('id', inplace=True)

  stop_type = 'bus'
  if folder.startswith('trenitalia'):
    stop_type = 'train'
  df_stops = df[['stop_name', 'place']].copy()
  df_stops.rename(columns={'stop_name': 'name'}, inplace=True)
  df_stops = df_stops.assign(type=stop_type)

  df_places = df[['stop_lat', 'stop_lon']].copy()
  df_places.rename(columns={'stop_lat': 'latitude', 'stop_lon': 'longitude'}, inplace=True)
  # p(df, df_places, df_stops)
  
  places = pd.concat([places.astype(df_places.dtypes), df_places])
  stops = pd.concat([stops.astype(df_stops.dtypes), df_stops])

places.index.set_names('id', inplace=True)
stops.index.set_names('id', inplace=True)

stops

,name,type,place
id,,,
1,Trento-Autostaz.,bus,1
2,Trento Solteri Bivio Ss.12,bus,2
3,Trento Bren Center,bus,3
4,Lavis,bus,4
5,S.Michele A/Adige,bus,5
...,...,...,...
ITRP-ST0180849001,PIZZALE-LUNGAVILLA F.S.,train,ITRP-ST0180849001
ITRP-ST0150859001,CORBETTA S.STEF. F.S.,train,ITRP-ST0150859001
ITRP-ST0080559001,SAN REMO F.S.,train,ITRP-ST0080559001


In [67]:
places

,address,latitude,longitude
id,,,
1,NaN,46.070785,11.118457
2,NaN,46.084046,11.119728
3,NaN,46.091322,11.115660
4,NaN,46.138362,11.109135
5,NaN,46.193685,11.132520
...,...,...,...
ITRP-ST0180849001,NaN,45.049837,9.068991
ITRP-ST0150859001,NaN,45.481200,8.918130
ITRP-ST0080559001,NaN,43.825007,7.782201


## Generate Agencies

In [68]:
AGENCIES_COLS = ['name', 'url']

agencies = DFS['agencies'].copy()
agencies.rename(columns={'agency_id': 'id', 'agency_name': 'name', 'agency_url': 'url'}, inplace=True)
agencies.set_index('id', inplace=True)

agencies

,name,url
id,,
1,Trentino trasporti S.p.A.,https://www.trentinotrasporti.it/
2,Trenitalia S.p.A.,https://www.trentinotrasporti.it/
3,Flixbus,https://www.flixbus.it/


## Generate Routes

Could exctract `bikes_allowed`, `trip_bikes_allowed`!! (for now removed)

In [69]:
ROUTES_COLS = ['provider', 'short_name', 'long_name', 'type']
routes = pd.DataFrame(columns=ROUTES_COLS)

TRIPS_COLS = ['route', 'headsign', 'direction', 'calendar', 'accessibility']
trips = pd.DataFrame(columns=TRIPS_COLS)

TRIPS_STOPS_COLS = ['trip', 'arrival_time', 'departure_time', 'stop', 'stop_sequence']
trips_stops = pd.DataFrame(columns=TRIPS_STOPS_COLS)

for folder in FOLDERS:
  df_routes = DFS[f'{folder}_routes'].copy()
  # p(df['route_type'].value_counts())
  
  agency_id = 0
  route_type = 'bus'
  if folder.startswith('tt_'): # Trentino trasporti dataset
    # Filter only busses (exclude funivia serdagna and potentially others)
    df_routes = (df_routes.loc[df_routes['route_type'] == 3])
    agency_id = 1
  elif folder.startswith('trenitalia'):
    agency_id = 2
    route_type = 'train'
  elif folder.startswith('flixbus'):
    agency_id = 3
  else:
    raise 'Agency not found for folder: ' + folder
  
  # Assign agency and type
  df_routes = df_routes.assign(agency_id=agency_id)
  df_routes = df_routes.assign(type=route_type)

  df_routes.rename(columns={'route_id': 'id', 'agency_id': 'provider', 'route_short_name': 'short_name', 'route_long_name': 'long_name'}, inplace=True)
  df_routes.drop(columns=['route_type', 'route_color', 'route_text_color', 'route_desc', 'route_url', 'bikes_allowed', 'route_sort_order'], inplace=True, errors='ignore')
  df_routes.set_index('id', inplace=True)
  for k in ['short_name']:
    if k in df_routes:
      df_routes[k] = df_routes[k].astype(str)

  df_trips = DFS[f'{folder}_trips'].copy()

  df_trips.rename(columns={'trip_id': 'id', 'route_id': 'route', 'service_id': 'calendar', 'trip_headsign': 'headsign', 'direction_id': 'direction', 'wheelchair_accessible': 'accessibility'}, inplace=True)
  df_trips.drop(columns=['shape_id', 'trip_short_name', 'route_short_name', 'block_id', 'trip_bikes_allowed', 'bikes_allowed', 'train_category', 'ticketing_trip_id', 'ticketing_type'], inplace=True, errors='ignore')
  if 'accessibility' in df_trips:
    df_trips['accessibility'] = df_trips['accessibility'].apply(lambda x: '0' if np.isnan(x) else '1')
    df_trips['accessibility'] = df_trips['accessibility'].astype(str)
  else:
    df_trips = df_trips.assign(accessibility='NaN')
  df_trips.set_index('id', inplace=True)
  if 'headsign' in df_trips:
    df_trips['headsign'] = df_trips['headsign'].astype(str)
  if 'direction' in df_trips:
    df_trips['direction'] = df_trips['direction'].astype(float)
  df_trips['calendar'] = df_trips['calendar'].astype(str)

  df_trips_stops = DFS[f'{folder}_stop_times'].copy()
  for k in ['trip_id', 'stop_id']:
    if k in df_trips_stops:
      df_trips_stops[k] = df_trips_stops[k].astype(str)
  df_trips_stops.rename(columns={'trip_id': 'trip', 'stop_id': 'stop'}, inplace=True)
  df_trips_stops.drop(columns=['timepoint', 'stop_headsign', 'route_short_name', 'pickup_type', 'drop_off_type', 'shape_dist_traveled'], inplace=True, errors='ignore')

  routes = pd.concat([routes.astype(df_routes.dtypes), df_routes])
  trips = pd.concat([trips.astype(df_trips.dtypes), df_trips])
  trips_stops = pd.concat([trips_stops.astype(df_trips_stops.dtypes), df_trips_stops])

In [70]:
routes

,provider,short_name,long_name,type
1,1,B101,Cavalese-Predazzo-Moena-Canazei-Penia,bus
3,1,B102,Trento - Cembra - Capriana - Cavalese,bus
7,1,B114,Casatta di Valfloriana - Palu' - Sicina,bus
55,1,B104,Trento-S.Michele a/A-Salorno-Egna-Ora-Cavalese,bus
60,1,B105,Verla - Valternigo - Palu' di Giovo,bus
...,...,...,...,...
ITRP-RO200_6570_20221211_20231209,2,6570,VENTIMIGLIA - CUNEO,train
ITRP-RO200_6436_20221211_20231209,2,6436,VENTIMIGLIA - TORINO P.NUOVA,train
ITRP-RO200_6437_20221211_20231209,2,6437,VENTIMIGLIA - TORINO P.NUOVA,train
ITRP-RO200_6502_20221211_20231209,2,6502,VOGHERA - ALESSANDRIA,train


In [71]:
trips

,route,headsign,direction,calendar,accessibility
0003126722023091120240621,565,Sopramonte,0.0,22023091120240621,NaN
0003111702023091120240621,336,Peio,0.0,4512023091120240621,NaN
0003067862023091120240621,136,Tesero,0.0,52023091120240621,NaN
0003067032023091120240621,1,Vigo di Fassa-Strada Neva,1.0,302023091120240621,NaN
0003103112023091120240621,522,Castello Tesino,0.0,62023091120240621,NaN
...,...,...,...,...,...
ITRP-TR200_6501_65973,ITRP-RO200_6501_20221211_20231209,ASTI F.S.,NaN,ITRP-TS200_6501_65973_20230610_20231209,NaN
ITRP-TR200_6501_65975,ITRP-RO200_6501_20221211_20231209,ASTI F.S.,NaN,ITRP-TS200_6501_65975_20230610_20231209,NaN
ITRP-TR200_6501_65977,ITRP-RO200_6501_20221211_20231209,ASTI F.S.,NaN,ITRP-TS200_6501_65977_20230610_20231209,NaN
ITRP-TR200_6501_65978,ITRP-RO200_6501_20221211_20231209,ASTI F.S.,NaN,ITRP-TS200_6501_65978_20230610_20231209,NaN


In [72]:
trips_stops

,trip,arrival_time,departure_time,stop,stop_sequence
0,0003061822023091120240621,07:35:00,07:35:00,721,1
1,0003061822023091120240621,07:36:00,07:36:00,720,2
2,0003061822023091120240621,07:37:00,07:37:00,719,3
3,0003061822023091120240621,07:39:00,07:39:00,718,4
4,0003061822023091120240621,07:41:00,07:41:00,2377,5
...,...,...,...,...,...
9095,ITRP-TR200_6501_63235,14:24:00,14:25:00,ITRP-ST0061639001,7
9096,ITRP-TR200_6501_63235,14:29:00,14:30:00,ITRP-ST0060689001,8
9097,ITRP-TR200_6501_63235,14:35:00,14:36:00,ITRP-ST0050969001,9
9098,ITRP-TR200_6501_63235,14:39:00,14:40:00,ITRP-ST0050289001,10


# Data filtering

In future data filtering on the final datasets will be performed here.

# Megre edu. facilities 

In [73]:
EDU_FILENAMES = ['high_school', 'middle_school'] # FIXME , 'university']

EDU_FAC_COLS = ['name', 'address', 'type']
edu_facilities = pd.DataFrame(columns=EDU_FAC_COLS)

for f in EDU_FILENAMES:
  df = pd.read_csv(f'{BASE_PATH}/edu/{f}.csv')
  df = df.astype(str)
  if 'id' in df.columns:
    df.set_index('id', inplace=True)
  # df.rename(columns={'trip_id': 'trip', 'stop_id': 'stop'}, inplace=True)
  df.drop(columns=['latitude', 'longitude'], inplace=True, errors='ignore')

  edu_facilities = pd.concat([edu_facilities.astype(df.dtypes), df])

edu_facilities

,name,address,type
TNSD020011,Liceo Arti Vittoria Bonporti Depero,Via delle Fosse 9 38068 Rovereto TN,highschool
TNSD010009,Giuseppe Soraperra,Strada Dolomites 6 38036 San Giovanni di Fass...,highschool
TNSD02000X,Liceo Arti Vittoria Bonporti Depero,Via Zambra 3 38121 Trento TN,highschool
TNPS00500B,Gardascuola,Via XXIV Maggio 1 38062 Arco TN,highschool
TNPS002011,Alcide Degasperi,Via XXIV Maggio 6 38051 Borgo Valsugana TN,highschool
...,...,...,...
TN1M007003,Associazione Pedagogica Rudolf Steiner,Via Conci 86 38123 Trento TN,middleschool
TNMM84401D,Vallelaghi-Vezzano Bellesini,Via Roma 3 38096 Vallelaghi TN,middleschool
TNMM846015,Villa Lagarina Anna Frank,Via Stockstadt Am Rhein 3 38060 Villa Lagarin...,middleschool
TNMM82902C,Ville d'Anuania - Tuenno,Via V. Maistrelli 11 38019 Ville d'Anaunia TN,middleschool


# Save DataFrames to files

In [74]:
OUT_BASE_PATH = '../Parsed data/'
DATAFRAMES = {
  # Transportation
  'agencies': agencies,
  'routes': routes,
  'trips': trips,
  'trip_calendars': calendars,
  'trip_calendar_exceptions': calendar_exceptions,
  'trip_stops': trips_stops,
  'stops': stops,
  'places': places,

  # Users
  'users': users,
  'user_commitments': users_commitments,
  'user_calendars': user_calendars,
  'user_calendar_exceptions': users_calendar_exceptions,

  # edu_facilities
  'educational_facilities': edu_facilities,
}

for name, df in DATAFRAMES.items():
  tmp = df.copy()
  cols = {}
  STATIC_RENAME = {
    'agencies': 'agency',
    'educational_facilities': 'educational_facility'
  }
  for c in df.columns:
    n = name[:-1]
    if name in STATIC_RENAME.keys():
      n = STATIC_RENAME[name]
    cols[c] = f'has_{n.capitalize()}_{c}'

  tmp = tmp.rename(columns=cols)
  tmp.index.set_names(f'has_{n.capitalize()}_id', inplace=True)

  # Write to file
  tmp.to_csv(f'{OUT_BASE_PATH}{name}.csv')